Grupo 3 - miniproyecto Deep Learning

# Datos y librerias

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [128]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import SGD, RMSprop, Nadam
from keras.optimizers.schedules import ExponentialDecay

In [167]:
# Carga de datos de archivo .csv
#dataTraining = pd.read_csv('trainReg.csv')
#dataTesting = pd.read_csv('testReg.csv')
# Carga de datos desde github
dataTraining = pd.read_csv('https://raw.githubusercontent.com/oapatinop/DL_miniproyecto/main/trainReg.csv')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/oapatinop/DL_miniproyecto/main/testReg.csv')

In [168]:
dataTraining = dataTraining.set_index('ID').drop('V3', axis=1)
dataTraining.head()

,Y,V1,V2,V4,V5,V6,V7,V8,V9,V10,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90
ID,,,,,,,,,,,,,,,,,,,,,
1,91,44.81144,0.83826,7.91314,10.94148,-0.04547,-15.16332,-10.47324,14.17212,10.57382,...,-8.44873,-230.30484,-40.94698,48.20025,-0.28694,155.76251,-56.23579,13.62599,123.92018,10.02845
2,76,41.99180,7.99976,64.26707,16.54115,-9.28737,-40.73524,33.60440,9.18802,-6.79144,...,18.68972,-44.06062,52.37792,81.36093,-14.81111,151.66273,-120.61213,10.57519,-3.21078,-1.07438
3,76,42.19196,2.23111,65.07719,24.99746,1.76100,6.66573,3.45778,-24.42779,-18.45069,...,-3.69878,-118.95712,54.15529,-23.32168,-9.65067,-83.83055,-141.17594,7.33084,-275.69714,2.35522
4,76,39.28634,-1.85716,91.04190,9.08333,0.08502,-5.59216,65.62463,8.33105,11.70269,...,20.89044,-18.53135,176.09769,351.33669,3.44682,121.69156,-270.43989,12.51659,-140.88884,-0.23476
5,76,40.36025,2.94918,53.83723,13.71369,-8.21964,-40.21636,21.22366,17.16742,0.14463,...,19.91979,34.59026,-69.83720,102.31946,8.08807,135.08089,-153.02327,4.09207,-68.33046,-6.19159


In [169]:
dataTesting = dataTesting.set_index('ID').drop('V3', axis=1)
dataTesting.head()

,V1,V2,V4,V5,V6,V7,V8,V9,V10,V11,...,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90
ID,,,,,,,,,,,,,,,,,,,,,
5972,35.74170,0.86615,1.44450,-13.87834,-9.57602,5.89475,-4.46036,-10.60501,-2.20779,-31.56674,...,10.78210,-183.63792,131.18868,37.68626,-4.48926,10.98139,-48.14549,-7.27992,-54.52826,4.81414
57116,35.31664,-0.47799,-12.27958,-29.32847,-2.38610,-7.27386,-1.87712,-3.74953,-3.38237,1.54788,...,53.92478,-238.77713,134.28149,81.19403,-4.81669,106.00178,3.34940,2.72328,156.94894,-8.44873
29076,43.25032,-6.31920,-2.56469,-7.59368,2.12862,23.29535,-0.34225,4.48761,4.93676,33.62086,...,40.95711,-269.59803,69.88747,29.65701,-0.12854,106.92919,-62.86550,30.12278,24.39843,-10.68316
83358,44.94220,-4.12303,-15.79205,7.15794,1.46007,-3.38280,-8.48270,12.69732,-1.68102,3.62442,...,0.81574,-243.09497,24.20334,-10.02719,19.25667,66.15602,-64.78646,9.09148,11.46055,-7.12136
84664,50.40813,-1.52061,-10.10142,-17.74876,-5.96768,-29.94136,-11.84237,5.90144,3.95996,6.07786,...,-3.58634,-61.67109,47.53814,139.04916,6.85904,-57.09356,-49.57683,-6.61453,22.55497,-4.39272


In [170]:
def guardar(ypred,nombre):
    ypred = pd.DataFrame(ypred,index=dataTesting.index, columns=['Y'])
    # Guardar predicciones en formato exigido en la competencia de kaggle
    archivo = "Resultados/"+ nombre + '.csv'
    ypred['Y'] = round(ypred['Y'])
    ypred.to_csv(archivo, index_label='ID')
    return ypred

In [171]:
def calculo_rmse(ypred,yTest,ypredTr,yTrain):
    RMSE = round(mse(np.round(ypred), yTest, squared = False),4)
    MSE = round(mse(np.round(ypred), yTest),4)
    #MAE = round(mae(ypred, yTest))

    RMSE_Tr = round(mse(np.round(ypredTr), yTrain, squared = False),4)
    MSE_Tr = round(mse(np.round(ypredTr), yTrain),4)
    #MAE_Tr = round(mae(ypredTr, yTrain))

    # Cálculo del desempeño del modelo utilizando el RMSE y el MAE
    print(f'El RMSE del modelo es de {RMSE}')
    print(f'El MSE del modelo es de {MSE}')
    #print(f'El MAE del modelo es de {MAE}')

    print(f'El RMSE del modelo en train es de {RMSE_Tr}')
    print(f'El MSE del modelo en train es de {MSE_Tr}')
    #print(f'El MAE del modelo en train es de {MAE_Tr}')

# Exploración de datos

In [172]:
dTr = dataTraining.copy()

In [173]:
dTr.describe(include="all").T

,count,mean,std,min,25%,50%,75%,max
Y,77779.0,80.308129,10.811038,4.00000,76.000000,84.00000,88.000000,92.00000
V1,77779.0,43.425185,6.128869,4.83688,40.060315,44.32385,47.900080,60.03401
V2,77779.0,-0.136720,4.370466,-69.68087,-2.612435,-0.06300,2.465950,23.81526
V4,77779.0,3.755313,17.609183,-165.22161,-7.016240,2.02210,12.776450,274.65858
V5,77779.0,-2.339768,14.483975,-121.47534,-10.685075,-2.05456,6.423900,160.81522
...,...,...,...,...,...,...,...,...
V86,77779.0,17.575097,115.243644,-3168.92457,-31.569390,15.23736,67.361475,2144.10391
V87,77779.0,-25.628857,173.310304,-4319.99232,-100.668480,-21.58164,51.333150,2833.60895
V88,77779.0,4.463484,13.526414,-236.03926,-2.569360,3.13690,10.002145,275.35366
V89,77779.0,18.664885,186.690183,-7458.37815,-60.121980,5.94149,84.389125,5289.11138


In [174]:
sum(dTr.isnull().sum() / dTr.shape[0] * 100)

0.0

# Preparación de datos

In [175]:
XTotal = dTr.loc[:,dTr.columns!='Y']
yTotal = dTr.loc[:,dTr.columns=='Y']

In [176]:
scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(XTotal)
XTotalS=scaler.transform(XTotal)
dTestS=scaler.transform(dataTesting)

In [177]:
XTrain, XTest, yTrain, yTest = train_test_split(XTotalS, yTotal, test_size=0.30, random_state=3)

# Análisis preliminar de modelos

## Keras

### Modelo Keras 2

In [68]:
knn2 = Sequential()
knn2.add(Dense(128, activation='relu', input_shape=(XTrain.shape[1],)))
knn2.add(BatchNormalization())
knn2.add(Dropout(0.2))
knn2.add(Dense(128, activation='relu'))
knn2.add(BatchNormalization())
knn2.add(Dropout(0.2))
knn2.add(Dense(128, activation='relu'))
knn2.add(Dense(1))
# Compilación del modelo
knn2.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamiento del modelo
knn2.fit(XTrain, yTrain.to_numpy(), epochs=250, batch_size=128)#, validation_split=0.2)

Epoch 1/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3081568.0000
Epoch 2/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 223840.8281
Epoch 3/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 63920.8047
Epoch 4/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 30102.3477
Epoch 5/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 21913.6426
Epoch 6/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 19519.5352
Epoch 7/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 18766.6738
Epoch 8/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 17803.9668
Epoch 9/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 17050.9883
Epoch 10/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 16178.2051
Epoch 11/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 15477.6465
Epoch 12/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 14663.1387
Epoch 13/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 14457.4434
Epoch 14/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - l

426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 94.7245
Epoch 79/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 92.0987
Epoch 80/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 89.0203
Epoch 81/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 88.4315
Epoch 82/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 87.9614
Epoch 83/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 87.9417
Epoch 84/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 84.4899
Epoch 85/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 82.2388
Epoch 86/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 168.2882
Epoch 87/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 82.8428
Epoch 88/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 83.4865
Epoch 89/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 83.4337
Epoch 90/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 82.9962
Epoch 91/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 82.5953
Epoch 92/250
426/426 ━━━━━━━━━━━

426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 68.8963
Epoch 157/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 69.7342
Epoch 158/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 69.3294
Epoch 159/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 68.6314
Epoch 160/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 69.4038
Epoch 161/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 68.8052
Epoch 162/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 68.5050
Epoch 163/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 68.1106
Epoch 164/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 68.2616
Epoch 165/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 69.9842
Epoch 166/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 69.8147
Epoch 167/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 67.7828
Epoch 168/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 68.0399
Epoch 169/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 69.2570
Epoch 170/250
426/42

426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 64.4765
Epoch 235/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 64.2164
Epoch 236/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 64.5467
Epoch 237/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 63.1245
Epoch 238/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 65.1387
Epoch 239/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 65.2616
Epoch 240/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 63.8198
Epoch 241/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 64.7133
Epoch 242/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 62.7257
Epoch 243/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 64.1632
Epoch 244/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 65.5329
Epoch 245/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 63.9477
Epoch 246/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 84.2448
Epoch 247/250
426/426 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 66.1573
Epoch 248/250
426/42

In [70]:
keras2=knn2.predict(XTest)
keras2Tr=knn2.predict(XTrain)
calculo_rmse(np.round(keras2),yTest,np.round(keras2Tr),yTrain)

730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 755us/step
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 1s 721us/step
El RMSE del modelo es de 8.6484
El MSE del modelo es de 74.7957
El RMSE del modelo en train es de 11.6791
El MSE del modelo en train es de 136.4002


In [71]:
kerasNN2=knn2.predict(dTestS)
guardar(kerasNN2,'KerasNN2_2')

584/584 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step


,Y
ID,
5972,2007.0
57116,1982.0
29076,2002.0
83358,2006.0
84664,2005.0
...,...
5763,2009.0
25441,2008.0
53318,1998.0


### Modelos keras 1

In [178]:
# Definición del modelo
knn = Sequential()
knn.add(Dense(128, input_dim=XTrain.shape[1], activation='sigmoid'))
knn.add(BatchNormalization())
knn.add(Dense(128, activation='sigmoid'))
knn.add(Dense(1))

# Compilación del modelo
knn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Entrenamiento del modelo
knn.fit(XTrain, yTrain.to_numpy(), epochs=150, batch_size=64)#, validation_split=0.2)

Epoch 1/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 3920.6929 - mean_squared_error: 3920.6929
Epoch 2/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 135.9171 - mean_squared_error: 135.9171
Epoch 3/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 90.8523 - mean_squared_error: 90.8523
Epoch 4/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 81.3842 - mean_squared_error: 81.3842
Epoch 5/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 81.9033 - mean_squared_error: 81.9033
Epoch 6/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 78.9325 - mean_squared_error: 78.9325
Epoch 7/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 78.7963 - mean_squared_error: 78.7963
Epoch 8/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 76.3112 - mean_squared_error: 76.3112
Epoch 9/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 74.7255 - mean_squared_error: 74.7255
Epoch 10/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 74.6369 - mean_squared_error: 74.6369
Epo

851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 37.9903 - mean_squared_error: 37.9903
Epoch 63/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 38.6887 - mean_squared_error: 38.6887
Epoch 64/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 38.0119 - mean_squared_error: 38.0119
Epoch 65/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 36.9902 - mean_squared_error: 36.9902
Epoch 66/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 36.5765 - mean_squared_error: 36.5765
Epoch 67/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 36.7582 - mean_squared_error: 36.7582
Epoch 68/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 36.5266 - mean_squared_error: 36.5266
Epoch 69/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 36.1693 - mean_squared_error: 36.1693
Epoch 70/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 35.3470 - mean_squared_error: 35.3470
Epoch 71/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 36.0549 - mean_squared_error: 36.0549
Epoch 72/150


851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 27.6859 - mean_squared_error: 27.6859
Epoch 124/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 27.6369 - mean_squared_error: 27.6369
Epoch 125/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 28.2135 - mean_squared_error: 28.2135
Epoch 126/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 27.7526 - mean_squared_error: 27.7526
Epoch 127/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 27.5801 - mean_squared_error: 27.5801
Epoch 128/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 27.4492 - mean_squared_error: 27.4492
Epoch 129/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 26.9772 - mean_squared_error: 26.9772
Epoch 130/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 27.2308 - mean_squared_error: 27.2308
Epoch 131/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 27.3840 - mean_squared_error: 27.3840
Epoch 132/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 26.9696 - mean_squared_error: 26.9696
Epoc

In [179]:
keras64=knn.predict(XTest)
keras64Tr=knn.predict(XTrain)
calculo_rmse(keras64,yTest,keras64Tr,yTrain)

730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 767us/step
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 1s 643us/step
El RMSE del modelo es de 9.6897
El MSE del modelo es de 93.8904
El RMSE del modelo en train es de 4.7343
El MSE del modelo en train es de 22.4131


In [180]:
keras64

array([[84.76213 ],
       [81.991875],
       [85.88212 ],
       ...,
       [78.213715],
       [80.46949 ],
       [75.82221 ]], dtype=float32)

### Modelo keras 3

In [146]:
knn3 = Sequential()
knn3.add(Dense(128, activation='relu', input_shape=(XTrain.shape[1],)))
knn3.add(BatchNormalization())
knn3.add(Dropout(0.2))
knn3.add(Dense(128, activation='softmax'))
knn3.add(BatchNormalization())
knn3.add(Dropout(0.2))
knn3.add(Dense(128, activation='softmax'))
knn3.add(Dense(1))
# Compilación del modelo
opt = Nadam(learning_rate=0.05)
knn3.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])

# Entrenamiento del modelo
knn3.fit(XTrain, yTrain.to_numpy(), epochs=150, batch_size=64)#, validation_split=0.2)

Epoch 1/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 3922262.7500 - mean_squared_error: 3922262.7500
Epoch 2/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3594379.7500 - mean_squared_error: 3594379.7500
Epoch 3/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3287722.0000 - mean_squared_error: 3287722.0000
Epoch 4/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2997750.5000 - mean_squared_error: 2997750.5000
Epoch 5/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2723384.0000 - mean_squared_error: 2723384.0000
Epoch 6/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2463671.5000 - mean_squared_error: 2463671.5000
Epoch 7/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2218377.5000 - mean_squared_error: 2218377.5000
Epoch 8/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1986700.6250 - mean_squared_error: 1986700.6250
Epoch 9/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1768734.6250 - mean_squared_error: 1768734.6250
Epoch 10/150
851/85

Epoch 60/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.8068 - mean_squared_error: 116.8068
Epoch 61/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 117.8524 - mean_squared_error: 117.8524
Epoch 62/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.0488 - mean_squared_error: 116.0488
Epoch 63/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.3068 - mean_squared_error: 116.3068
Epoch 64/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 115.5301 - mean_squared_error: 115.5301
Epoch 65/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.8763 - mean_squared_error: 116.8763
Epoch 66/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 117.5710 - mean_squared_error: 117.5710
Epoch 67/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 118.7240 - mean_squared_error: 118.7240
Epoch 68/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 337.0384 - mean_squared_error: 337.0384
Epoch 69/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.5267 - mean_squa

851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.9060 - mean_squared_error: 116.9060
Epoch 121/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 117.4260 - mean_squared_error: 117.4260
Epoch 122/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.9111 - mean_squared_error: 116.9111
Epoch 123/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.9680 - mean_squared_error: 116.9680
Epoch 124/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.4450 - mean_squared_error: 116.4450
Epoch 125/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 114.6846 - mean_squared_error: 114.6846
Epoch 126/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.0728 - mean_squared_error: 116.0728
Epoch 127/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 117.3016 - mean_squared_error: 117.3016
Epoch 128/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 117.3188 - mean_squared_error: 117.3188
Epoch 129/150
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 116.4934 - mean_squared_

In [147]:
keras3=knn3.predict(XTest)
keras3Tr=knn3.predict(XTrain)
calculo_rmse(np.round(keras3),yTest,np.round(keras3Tr),yTrain)

730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 971us/step
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 1s 835us/step
El RMSE del modelo es de 18.1343
El MSE del modelo es de 328.8531
El RMSE del modelo en train es de 20.7476
El MSE del modelo en train es de 430.4615


In [34]:
kerasNN3=knn.predict(dTestS)
guardar(kerasNN3,'KerasNN3')

584/584 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step


,Y
ID,
5972,2000.0
57116,1991.0
29076,1993.0
83358,2001.0
84664,2004.0
...,...
5763,2010.0
25441,2002.0
53318,1999.0


## Scikit Learn

### Scikit learn 1

In [72]:
sklnn = MLPRegressor(hidden_layer_sizes=(128, 128, 128, 128), activation='logistic', solver='lbfgs',
                     max_iter=250, random_state=3)
sklnn.fit(XTrain, yTrain)

MLPRegressor(activation='logistic', hidden_layer_sizes=(128, 128, 128, 128),
             max_iter=250, random_state=3, solver='lbfgs')

In [73]:
sklNN=sklnn.predict(XTest)
sklNNTr=sklnn.predict(XTrain)
calculo_rmse(sklNN,yTest,sklNNTr,yTrain)

El RMSE del modelo es de 8.785
El MSE del modelo es de 77.1755
El RMSE del modelo en train es de 8.5798
El MSE del modelo en train es de 73.6128


In [98]:
sklNN1L=sklnn.predict(dTestS)
guardar(sklNN1L,'Skl_NN1L')

,Y
ID,
5972,2007.0
57116,1976.0
29076,1997.0
83358,2006.0
84664,2004.0
...,...
5763,2009.0
25441,2008.0
53318,1996.0


## Combinado

In [95]:
combinado = (sklNN + np.reshape(keras2, -1))/2
combinadoTr = (sklNNTr + np.reshape(keras2Tr, -1))/2
calculo_rmse(combinado,yTest,combinadoTr,yTrain)

El RMSE del modelo es de 8.57
El MSE del modelo es de 73.4457
El RMSE del modelo en train es de 9.0814
El MSE del modelo en train es de 82.4715


In [97]:
sklNN1L + kerasNN2
combinadoT = (sklNN1L + np.reshape(kerasNN2, -1))/2
guardar(combinadoT,'CombinadoSK1_keras2')

,Y
ID,
5972,2007.0
57116,1979.0
29076,2000.0
83358,2006.0
84664,2004.0
...,...
5763,2009.0
25441,2008.0
53318,1997.0


### Scikit learn 2

In [110]:
sklnn2 = MLPRegressor(hidden_layer_sizes=(128, 128, 128, 128, 128), activation='logistic', solver='adam',
                     batch_size=128, max_iter=250, random_state=3)
sklnn2.fit(XTrain, yTrain)

MLPRegressor(activation='logistic', batch_size=128,
             hidden_layer_sizes=(128, 128, 128, 128, 128), max_iter=250,
             random_state=3)

In [111]:
sklNN2=sklnn2.predict(XTest)
sklNNTr2=sklnn2.predict(XTrain)
calculo_rmse(sklNN2,yTest,sklNNTr2,yTrain)

El RMSE del modelo es de 10.3392
El MSE del modelo es de 106.8996
El RMSE del modelo en train es de 5.1223
El MSE del modelo en train es de 26.2375


In [ ]:
#sklNN2RA=sklnn2.predict(dTestS)
#guardar(sklNN2RA,'Skl_NN1L')

## Random Search Scikit learn

In [ ]:
param_test = {
    'hidden_layer_sizes':[(64,64),(128,128,128),(256,256)],
    'activation':['identity', 'logistic', 'tanh', 'relu'],
    'solver':['lbfgs', 'sgd', 'adam']
}
RS_skl = RandomizedSearchCV(estimator = MLPRegressor(max_iter=200, random_state=3),
                        param_distributions = param_test, scoring='neg_mean_squared_error',n_iter=5,n_jobs=-1, cv=5, verbose=True)
RS_skl.fit(XTrain,yTrain)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5, estimator=MLPRegressor(random_state=3), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'activation': ['identity', 'logistic',
                                                       'tanh', 'relu'],
                                        'hidden_layer_sizes': [(64, 64),
                                                               (128, 128, 128),
                                                               (256, 256)],
                                        'solver': ['lbfgs', 'sgd', 'adam']},
                   scoring='neg_mean_squared_error', verbose=True)

In [ ]:
RS_skl.best_params_ , RS_skl.best_score_

({'solver': 'lbfgs', 'hidden_layer_sizes': (64, 64), 'activation': 'logistic'},
 -87.07201952336706)

In [116]:
sklnn3 = MLPRegressor(hidden_layer_sizes=(64, 64, 64, 64, 64), activation='logistic', solver='lbfgs',
                     max_iter=200, random_state=3)
sklnn3.fit(XTrain, yTrain)
sklNN3=sklnn3.predict(XTest)
sklNNTr3=sklnn3.predict(XTrain)
calculo_rmse(sklNN3,yTest,sklNNTr3,yTrain)

El RMSE del modelo es de 8.8098
El MSE del modelo es de 77.6127
El RMSE del modelo en train es de 8.65
El MSE del modelo en train es de 74.8232


In [22]:
sklNN3T=sklnn3.predict(dTestS)
guardar(sklNN3T,'Skl_NNRS3')

,Y
ID,
5972,2003.572287
57116,1971.812773
29076,2005.042888
83358,2003.109432
84664,2006.437235
...,...
5763,2009.757227
25441,2002.872724
53318,2002.224477


# Entrenamiento y calibración

In [ ]:
predNN1=sklnn.predict(dTestS)
guardar(predNN1,'Skl_NN4')

,Y
ID,
5972,2005.826770
57116,1992.707868
29076,1999.214673
83358,2001.212872
84664,2005.111924
...,...
5763,2011.702906
25441,2000.864693
53318,1997.623429


# Visualización de resultados